In [43]:
# imports
import requests
import pandas as pd
import json
from IPython.display import JSON 

In [44]:
bike_df = pd.read_csv("bike_df_austin.csv")
bike_df

,Station Name,Latitude,Longitude,Number of Bikes,Empty Slots,ll
0,9th/Henderson,30.27217,-97.75246,5,8,"30.27217,-97.75246"
1,South Congress @ Bouldin Creek,30.25495,-97.74755,3,8,"30.25495,-97.74755"
2,21st/University,30.28354,-97.73953,0,18,"30.28354,-97.73953"
3,Lake Austin Blvd/Deep Eddy,30.27807,-97.77272,7,4,"30.27807,-97.77272"
4,Sterzing/Barton Springs,30.26406,-97.76385,2,7,"30.26406,-97.76385"
...,...,...,...,...,...,...
76,South Congress/Elizabeth,30.24891,-97.75019,4,7,"30.24891,-97.75019"
77,4th/Guadalupe @ Republic Square,30.26727,-97.74718,5,10,"30.26727,-97.74718"
78,South 1st/Riverside @ Long Center,30.25941,-97.74971,10,1,"30.25941,-97.74971"
79,Dean Keeton/Park Place,30.28928,-97.73300,7,2,"30.28928,-97.733"


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [45]:
# Only did 150 meters as otherwise stations cross eachother and there is too many overlaps of the same POIs per location.
full_foursquare_data = []

for index, row in bike_df.iterrows():

    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "ll" : row["ll"],
        "categories" : "13003,10027,13065",
        "radius" : "150",
        "sort":"DISTANCE",
        "limit" : "50",
        "fields" : "name,geocodes,categories,price,rating,popularity,distance"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": "fsq3qnvtw345lsOhc+snv7hneg8d02DFLOe2i9PR0u82sp0="
    }

    response = requests.request("GET", url, params=params, headers=headers)

    data4 = response.json()

    full_foursquare_data.append(data4)






Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [46]:
counter = 0
foursquare_rows = []
for response in full_foursquare_data:
    restaurants = response["results"]
    ll = bike_df["ll"][counter]
    counter +=1
    for restaurant in restaurants:
        data = {}
        data["name"] = restaurant["name"]
        data["category"] = restaurant["categories"][0]["name"]
        data["latitude"] = restaurant["geocodes"]["main"]["latitude"]
        data["longitude"] = restaurant["geocodes"]["main"]["longitude"]
        data["distance"] = restaurant["distance"]
        data["bike_location"] = ll

        try:
            data["rating"] = restaurant["rating"]
        except KeyError:
            data["rating"] = None
        try:
            data["popularity"] = restaurant["popularity"]
        except KeyError:
            data["popularity"] = None
        try:
            data["price"] = restaurant["price"]
        except KeyError:
            data["price"] = None
        foursquare_rows.append(data)

Put your parsed results into a DataFrame

In [47]:
foursquaredf = pd.DataFrame(foursquare_rows)

In [49]:
foursquaredf.to_csv("df_austin_foursquare_data.csv", index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [50]:
# Only did 150 meters as otherwise stations cross eachother and there is too many overlaps of the same POIs per location.

full_yelp_data = []
api_key = "q5Ks_s0oQ8NWcDRMGsa3Ca7B854LZ9NkRrYkcRwOKRAe-Si3YUD6ReCs10sSC3YXnWDCcdypoj5Scl6jjO75W2DwMhAl0lIj6KeKwSSd51w2k0TbXhhqhIFByubVY3Yx"
headers = {
    "Authorization": f"Bearer {api_key}"
}
parameters = {
    "term": "restaurant,bar",
    "radius": 150,
    "latitude": None,
    "longitude": None,
}

for index, row in bike_df.iterrows():
    parameters["latitude"] = row["Latitude"]
    parameters["longitude"] = row["Longitude"]
    yresponse = requests.get("https://api.yelp.com/v3/businesses/search", headers=headers, params=parameters)
    data = yresponse.json()

    full_yelp_data.append(data)






Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [65]:
counter = 0
yelp_rows = []
for yresponse in full_yelp_data:
    yrestaurants = yresponse["businesses"]
    ll = bike_df["ll"][counter]
    counter +=1
    for yrestaurant in yrestaurants:
        ydata = {}
        ydata["name"] = yrestaurant["name"]
        ydata["category"] = yrestaurant["categories"][0]["title"]
        ydata["latitude"] = yrestaurant["coordinates"]["latitude"]
        ydata["longitude"] = yrestaurant["coordinates"]["longitude"]
        ydata["distance"] = yrestaurant["distance"]
        ydata["bike_location"] = ll
        ydata["review_count"] = yrestaurant["review_count"]

        try:
            ydata["rating"] = yrestaurant["rating"]
        except KeyError:
            ydata["rating"] = None
        try:
            ydata["popularity"] = yrestaurant["popularity"]
        except KeyError:
            ydata["popularity"] = None
        try:
            ydata["price"] = yrestaurant["price"]
        except KeyError:
            ydata["price"] = None
        yelp_rows.append(ydata)

In [57]:
full_yelp_data[1]

{'businesses': [{'id': 'TBsxo5lJvwgmKo1aRTTJrg',
   'alias': 'prim-and-proper-austin',
   'name': 'Prim and Proper',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/foX4HK_p-AG1UNYMO4aFPA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/prim-and-proper-austin?adjust_creative=29zC3K-Gh-e9CGAFP_YpFA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=29zC3K-Gh-e9CGAFP_YpFA',
   'review_count': 37,
   'categories': [{'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
    {'alias': 'coffee', 'title': 'Coffee & Tea'},
    {'alias': 'tradamerican', 'title': 'American (Traditional)'}],
   'rating': 4.5,
   'coordinates': {'latitude': 30.254224644988437,
    'longitude': -97.74761894020668},
   'transactions': ['delivery', 'pickup'],
   'price': '$$',
   'location': {'address1': '1007 S Congress Ave',
    'address2': 'Ste 120',
    'address3': '',
    'city': 'Austin',
    'zip_code': '78704',
    'country': 'US',
    'state': 'TX',
    'display_ad

Put your parsed results into a DataFrame

In [66]:
yelpdf = pd.DataFrame(yelp_rows)
yelpdf
#returns odd error where every other row is a duplicate of non requested data with columns I never requested. Unsure why...

,name,category,latitude,longitude,distance,bike_location,review_count,rating,popularity,price
0,Prim and Proper,Cocktail Bars,30.254225,-97.747619,80.927065,"30.25495,-97.74755",37,4.5,None,$$
1,Freebirds World Burrito,Mexican,30.255994,-97.746911,131.363290,"30.25495,-97.74755",229,3.5,None,$
2,Pool Burger,Tiki Bars,30.277800,-97.772590,30.718116,"30.27807,-97.77272",158,3.5,None,$$
3,68 Degrees Kitchen,Italian,30.277860,-97.772900,30.066667,"30.27807,-97.77272",149,3.5,None,$$
4,Tumble 22,American (Traditional),30.277925,-97.771737,95.770277,"30.27807,-97.77272",227,4.0,None,$$
...,...,...,...,...,...,...,...,...,...,...
485,Guild City Center,Hotels,30.266180,-97.748074,148.538343,"30.26727,-97.74718",38,4.0,None,$$
486,Aussie's Grill & Beach Bar,Bars,30.258980,-97.748080,149.172946,"30.25941,-97.74971",204,3.0,None,$$
487,Sandy's Hamburgers,Burgers,30.258400,-97.750990,172.833799,"30.25941,-97.74971",493,3.5,None,$
488,Texas Monthly BBQ Festival,Festivals,30.260690,-97.750520,162.490598,"30.25941,-97.74971",15,4.5,None,None


In [67]:
yelpdf.to_csv("df_austin_yelp_data.csv", index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [71]:
df_top10_foursquare = foursquaredf.sort_values(by="rating",ascending=False)
top_10_foursquare = df_top10_foursquare[:10]
top_10_foursquare

,name,category,latitude,longitude,distance,bike_location,rating,popularity,price
764,Hopdoddy Burger Bar,Bar,30.249610,-97.749881,86,"30.24891,-97.75019",9.3,0.983106,2.0
762,Home Slice Pizza,Pizzeria,30.249288,-97.749577,42,"30.24891,-97.75019",9.3,0.997452,2.0
149,Harry Ransom Center,Art Museum,30.284141,-97.741193,91,"30.28395,-97.74198",9.2,0.960649,NaN
416,Mozart's Coffee Roasters,Bakery,30.295315,-97.784270,131,"30.29439,-97.78375",9.2,0.998338,1.0
575,Suerte,Mexican Restaurant,30.262175,-97.723141,112,"30.2627,-97.7243",9.2,0.981325,1.0
83,Fareground,Restaurant,30.263465,-97.744268,86,"30.26408,-97.74355",9.1,0.994920,2.0
345,True Food Kitchen,Restaurant,30.267354,-97.752224,71,"30.2678,-97.75189",9.1,0.976079,3.0
63,Red Ash,Italian Restaurant,30.265807,-97.744940,141,"30.26634,-97.74378",9.1,0.992309,2.0
431,Fareground,Restaurant,30.263465,-97.744268,91,"30.26332,-97.74508",9.1,0.994920,2.0
499,Greater Goods Coffee Roasters,Café,30.257748,-97.715064,127,"30.25902,-97.71479",9.1,0.986880,1.0


In [72]:
df_top10_yelp = yelpdf.sort_values(by="rating",ascending=False)
top_10_yelp = df_top10_yelp[:10]
top_10_yelp

,name,category,latitude,longitude,distance,bike_location,review_count,rating,popularity,price
148,International Grub,Food Trucks,30.261900,-97.761670,60.611184,"30.262,-97.76118",10,5.0,None,None
476,Gelato Paradiso,Desserts,30.249770,-97.750120,95.863624,"30.24891,-97.75019",101,5.0,None,$$
470,Lobby Bar,Bars,30.247746,-97.749874,132.937453,"30.24891,-97.75019",1,5.0,None,None
121,Kimber Modern,Hotels,30.250104,-97.748397,132.187426,"30.25103,-97.74926",157,5.0,None,$$$
332,Atmosphere Coworking,Shared Office Spaces,30.254961,-97.717568,93.333108,"30.25544,-97.71677",4,5.0,None,None
22,Out of Nowhere ATX,Food Trucks,30.265690,-97.743940,72.538064,"30.26634,-97.74378",15,5.0,None,None
196,DittyDog,Hot Dogs,30.267396,-97.736912,134.406319,"30.26854,-97.73646",58,5.0,None,None
10,Barton Springs Bike Rental & Tours,Bike Rentals,30.262926,-97.762963,152.169432,"30.26406,-97.76385",309,5.0,None,$$
109,The Frances Modern,Hotels,30.268729,-97.728389,25.991347,"30.26896,-97.72843",20,5.0,None,$$$
201,Apt 115,Wine Bars,30.261600,-97.719820,163.132690,"30.26032,-97.71899",78,5.0,None,$$
